# GloVe from the Stanford NLP Group

### Turning GloVe text file to data structure (dictionary)

In [1]:
f = open('glove.6B/glove.6B.50d.txt','r')
model = {}
for line in f:
    split_line = line.split()
    word = split_line[0]
    embedding = [float(val) for val in split_line[1:]]
    model[word] = embedding

In [3]:
model['hello']

[-0.38497,
 0.80092,
 0.064106,
 -0.28355,
 -0.026759,
 -0.34532,
 -0.64253,
 -0.11729,
 -0.33257,
 0.55243,
 -0.087813,
 0.9035,
 0.47102,
 0.56657,
 0.6985,
 -0.35229,
 -0.86542,
 0.90573,
 0.03576,
 -0.071705,
 -0.12327,
 0.54923,
 0.47005,
 0.35572,
 1.2611,
 -0.67581,
 -0.94983,
 0.68666,
 0.3871,
 -1.3492,
 0.63512,
 0.46416,
 -0.48814,
 0.83827,
 -0.9246,
 -0.33722,
 0.53741,
 -1.0616,
 -0.081403,
 -0.67111,
 0.30923,
 -0.3923,
 -0.55002,
 -0.68827,
 0.58049,
 -0.11626,
 0.013139,
 -0.57654,
 0.048833,
 0.67204]

In [4]:
import numpy as np
import pylab as Plot

### Converting dictionary to matrix

In [5]:
# glove_matrix = Math.loadtxt("glove.6B.300d.embeddings.txt");
# glove_words = [line.strip() for line in open("glove.6B.300d.labels.txt")]
# glove_matrix = mod.values()
# glove_words = mod.keys()

In [6]:
target_words = [line.strip().lower() for line in open("1000-words-csv.csv")]

rows = [model.keys().index(word) for word in target_words if word in model.keys()]

In [7]:
rows[:10]

[365155, 122470, 358154, 271563, 341134, 19225, 19212, 19215, 41830, 91090]

In [8]:
target_matrix = np.array([model.values()[i] for i in rows])
#for i in rows:
#    target_matrix.append(glove_matrix[i])

In [9]:
target_matrix[:2]

array([[  4.18000000e-01,   2.49680000e-01,  -4.12420000e-01,
          1.21700000e-01,   3.45270000e-01,  -4.44570000e-02,
         -4.96880000e-01,  -1.78620000e-01,  -6.60230000e-04,
         -6.56600000e-01,   2.78430000e-01,  -1.47670000e-01,
         -5.56770000e-01,   1.46580000e-01,  -9.50950000e-03,
          1.16580000e-02,   1.02040000e-01,  -1.27920000e-01,
         -8.44300000e-01,  -1.21810000e-01,  -1.68010000e-02,
         -3.32790000e-01,  -1.55200000e-01,  -2.31310000e-01,
         -1.91810000e-01,  -1.88230000e+00,  -7.67460000e-01,
          9.90510000e-02,  -4.21250000e-01,  -1.95260000e-01,
          4.00710000e+00,  -1.85940000e-01,  -5.22870000e-01,
         -3.16810000e-01,   5.92130000e-04,   7.44490000e-03,
          1.77780000e-01,  -1.58970000e-01,   1.20410000e-02,
         -5.42230000e-02,  -2.98710000e-01,  -1.57490000e-01,
         -3.47580000e-01,  -4.56370000e-02,  -4.42510000e-01,
          1.87850000e-01,   2.78490000e-03,  -1.84110000e-01,
        

In [8]:
#target_matrix = np.array(target_matrix)

### Using a TSNE algorithm by Laurens van der Maaten

- t-Distributed Stochastic Neighbor Embedding

In [11]:
#
#  tsne.py
#
# Implementation of t-SNE in Python. The implementation was tested on Python 2.7.10, and it requires a working
# installation of NumPy. The implementation comes with an example on the MNIST dataset. In order to plot the
# results of this example, a working installation of matplotlib is required.
#
# The example can be run by executing: `ipython tsne.py`
#
#
#  Created by Laurens van der Maaten on 20-12-08.
#  Copyright (c) 2008 Tilburg University. All rights reserved.

import numpy as Math
import pylab as Plot

def Hbeta(D = Math.array([]), beta = 1.0):
	"""Compute the perplexity and the P-row for a specific value of the precision of a Gaussian distribution."""

	# Compute P-row and corresponding perplexity
	P = Math.exp(-D.copy() * beta);
	sumP = sum(P);
	H = Math.log(sumP) + beta * Math.sum(D * P) / sumP;
	P = P / sumP;
	return H, P;


def x2p(X = Math.array([]), tol = 1e-5, perplexity = 30.0):
	"""Performs a binary search to get P-values in such a way that each conditional Gaussian has the same perplexity."""

	# Initialize some variables
	print "Computing pairwise distances..."
	(n, d) = X.shape;
	sum_X = Math.sum(Math.square(X), 1);
	D = Math.add(Math.add(-2 * Math.dot(X, X.T), sum_X).T, sum_X);
	P = Math.zeros((n, n));
	beta = Math.ones((n, 1));
	logU = Math.log(perplexity);

	# Loop over all datapoints
	for i in range(n):

		# Print progress
		if i % 500 == 0:
			print "Computing P-values for point ", i, " of ", n, "..."

		# Compute the Gaussian kernel and entropy for the current precision
		betamin = -Math.inf;
		betamax =  Math.inf;
		Di = D[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))];
		(H, thisP) = Hbeta(Di, beta[i]);

		# Evaluate whether the perplexity is within tolerance
		Hdiff = H - logU;
		tries = 0;
		while Math.abs(Hdiff) > tol and tries < 50:

			# If not, increase or decrease precision
			if Hdiff > 0:
				betamin = beta[i].copy();
				if betamax == Math.inf or betamax == -Math.inf:
					beta[i] = beta[i] * 2;
				else:
					beta[i] = (beta[i] + betamax) / 2;
			else:
				betamax = beta[i].copy();
				if betamin == Math.inf or betamin == -Math.inf:
					beta[i] = beta[i] / 2;
				else:
					beta[i] = (beta[i] + betamin) / 2;

			# Recompute the values
			(H, thisP) = Hbeta(Di, beta[i]);
			Hdiff = H - logU;
			tries = tries + 1;

		# Set the final row of P
		P[i, Math.concatenate((Math.r_[0:i], Math.r_[i+1:n]))] = thisP;

	# Return final P-matrix
	print "Mean value of sigma: ", Math.mean(Math.sqrt(1 / beta));
	return P;


def pca(X = Math.array([]), no_dims = 50):
	"""Runs PCA on the NxD array X in order to reduce its dimensionality to no_dims dimensions."""

	print "Preprocessing the data using PCA..."
	(n, d) = X.shape;
	X = X - Math.tile(Math.mean(X, 0), (n, 1));
	(l, M) = Math.linalg.eig(Math.dot(X.T, X));
	Y = Math.dot(X, M[:,0:no_dims]);
	return Y;


def tsne(X = Math.array([]), no_dims = 2, initial_dims = 50, perplexity = 30.0):
	"""Runs t-SNE on the dataset in the NxD array X to reduce its dimensionality to no_dims dimensions.
	The syntaxis of the function is Y = tsne.tsne(X, no_dims, perplexity), where X is an NxD NumPy array."""

	# Check inputs
	if isinstance(no_dims, float):
		print "Error: array X should have type float.";
		return -1;
	if round(no_dims) != no_dims:
		print "Error: number of dimensions should be an integer.";
		return -1;

	# Initialize variables
	X = pca(X, initial_dims).real;
	(n, d) = X.shape;
	max_iter = 1000;
	initial_momentum = 0.5;
	final_momentum = 0.8;
	eta = 500;
	min_gain = 0.01;
	Y = Math.random.randn(n, no_dims);
	dY = Math.zeros((n, no_dims));
	iY = Math.zeros((n, no_dims));
	gains = Math.ones((n, no_dims));

	# Compute P-values
	P = x2p(X, 1e-5, perplexity);
	P = P + Math.transpose(P);
	P = P / Math.sum(P);
	P = P * 4;									# early exaggeration
	P = Math.maximum(P, 1e-12);

	# Run iterations
	for iter in range(max_iter):

		# Compute pairwise affinities
		sum_Y = Math.sum(Math.square(Y), 1);
		num = 1 / (1 + Math.add(Math.add(-2 * Math.dot(Y, Y.T), sum_Y).T, sum_Y));
		num[range(n), range(n)] = 0;
		Q = num / Math.sum(num);
		Q = Math.maximum(Q, 1e-12);

		# Compute gradient
		PQ = P - Q;
		for i in range(n):
			dY[i,:] = Math.sum(Math.tile(PQ[:,i] * num[:,i], (no_dims, 1)).T * (Y[i,:] - Y), 0);

		# Perform the update
		if iter < 20:
			momentum = initial_momentum
		else:
			momentum = final_momentum
		gains = (gains + 0.2) * ((dY > 0) != (iY > 0)) + (gains * 0.8) * ((dY > 0) == (iY > 0));
		gains[gains < min_gain] = min_gain;
		iY = momentum * iY - eta * (gains * dY);
		Y = Y + iY;
		Y = Y - Math.tile(Math.mean(Y, 0), (n, 1));

		# Compute current value of cost function
		if (iter + 1) % 10 == 0:
			C = Math.sum(P * Math.log(P / Q));
			print "Iteration ", (iter + 1), ": error is ", C

		# Stop lying about P-values
		if iter == 100:
			P = P / 4;

	# Return solution
	return Y;


#if __name__ == "__main__":
#	print "Run Y = tsne.tsne(X, no_dims, perplexity) to perform t-SNE on your dataset."
#	print "Running example on 2,500 MNIST digits..."
#	X = Math.loadtxt("mnist2500_X.txt");
#	labels = Math.loadtxt("mnist2500_labels.txt");
#	Y = tsne(X, 2, 50, 20.0);
#	Plot.scatter(Y[:,0], Y[:,1], 20, labels);
#	Plot.show();


In [12]:
reduced_matrix = tsne(target_matrix, 2)

Preprocessing the data using PCA...
Computing pairwise distances...
Computing P-values for point  0  of  995 ...
Computing P-values for point  500  of  995 ...
Mean value of sigma:  1.28544506544
Iteration  10 : error is  18.1052019562
Iteration  20 : error is  17.7098824514
Iteration  30 : error is  17.3119348526
Iteration  40 : error is  17.4729267115
Iteration  50 : error is  17.3254802149
Iteration  60 : error is  17.3816951498
Iteration  70 : error is  17.3635778577
Iteration  80 : error is  17.3610285778
Iteration  90 : error is  17.3585244808
Iteration  100 : error is  17.3384517717
Iteration  110 : error is  2.2119993211
Iteration  120 : error is  1.83787400658
Iteration  130 : error is  1.70013163273
Iteration  140 : error is  1.62307522607
Iteration  150 : error is  1.57301050383
Iteration  160 : error is  1.5380366567
Iteration  170 : error is  1.51245646747
Iteration  180 : error is  1.49264595899
Iteration  190 : error is  1.47705183578
Iteration  200 : error is  1.4645690

In [13]:
import matplotlib.pyplot as plt

In [18]:
plt.figure(figsize=(100, 100), dpi=100)
max_x = Math.amax(reduced_matrix, axis=0)[0]
max_y = Math.amax(reduced_matrix, axis=0)[1]
plt.xlim((-max_x,max_x))
plt.ylim((-max_y,max_y))

plt.scatter(reduced_matrix[:, 0], reduced_matrix[:, 1], 20);

for row_id in range(0, len(rows)):
    target_word = model.keys()[rows[row_id]]
    x = reduced_matrix[row_id, 0]
    y = reduced_matrix[row_id, 1]
    plt.annotate(target_word, (x,y), fontsize=20)

plt.savefig("glove_1000.png")